# Query a Table with Apache Cassandra

In [22]:
# Import Apache Cassandra

import cassandra


## Let's create a connection to the databasse


In [23]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1']) # If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

### Let's create a keyspace to do our work in


In [24]:
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
                   )
except Exception as e:
    print(e)

### Connect to the keyspace. Compare this to how we had create a new session in PostgreSQL.

In [25]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Unlike RDBM, We can't model our data and create our table with out more information NoSQL 

## What queries will I be performing on this data?


##### In this case I would like to be able to get every album that was released in a particular year.

```bash
select * from music_library WHERE year=1970
```
2. Give me every album in my music library that was created by a given artist

`select * from artist_library WHERE artist_name='The Beatles'`

### Because of this I need to be able to do a WHERE on YEAR. YEAR will become my partition key, and artist name will be my clustering column to make each Primary key unique. Remember there are no duplicate in Apache Cassandra.

In [26]:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "Create Table if not exists album_library"
query = query + "(year int, artist_name text, album_name text, primary key (artist_name, year))"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Let's insert some data into both tables



In [31]:
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + "VALUES (%s, %s, %s)"

query1 = "INSERT INTO album_library (artist_name, year, album_name)"
query1 = query1 + "VALUES (%s, %s, %s)"


try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation"))
except Exception as e:
    print(e)

    
try:
    session.execute(query, (1966, "The Monkees", "The Monkees"))
except Exception as e:
    print(e)
    
    
try:
    session.execute(query, (1970, "The Carpenters", "Close To You"))
except Exception as e:
    print(e)

    
try:
    session.execute(query1, ("The Beatles", 1965, "Rubber Soul"))
except Exception as e:
    print(e)
    

try:
    session.execute(query1, ("The Beatles", 1970, "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Who", 1965, "My Generation"))
except Exception as e:
    print(e)

    
try:
    session.execute(query1, ("The Monkees", 1966, "The Monkees"))
except Exception as e:
    print(e)
    
    
try:
    session.execute(query1, ("The Carpenters", 1970, "Close To You"))
except Exception as e:
    print(e) 


### Validate if the data was inserted 



In [32]:
query = 'SELECT * from music_library'

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:  # the for loop is for printing, it will not be required if executing in cqlsh
    print(row.year, row.album_name, row.artist_name)

1965 Rubber Soul The Beatles
1965 My Generation The Who
1970 Let it Be The Beatles
1970 Close To You The Carpenters
1966 The Monkees The Monkees


In [33]:
query = "SELECT * from album_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.year, row.album_name)

The Carpenters 1970 Close To You
The Beatles 1965 Rubber Soul
The Beatles 1970 Let it Be
The Who 1965 My Generation
The Monkees 1966 The Monkees


### Let's Validate our data model with our original query

select * from music_library WHERE year=1970

In [34]:
query = "select * from music_library WHERE year=1970"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:  # the for loop is for printing, it will not be required if executing in cqlsh
    print(row.year, row.artist_name, row.album_name)

1970 The Beatles Let it Be
1970 The Carpenters Close To You


`select * from artist_library WHERE artist_name='The Beatles'`

In [35]:
query = "select * from album_library WHERE artist_name='The Beatles'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:  # the for loop is for printing, it will not be required if executing in cqlsh
    print(row.artist_name, row.year, row.album_name)

The Beatles 1965 Rubber Soul
The Beatles 1970 Let it Be


### For the sake of the demo, let's drop the table

In [ ]:
query = "drop table music_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table album_library"

try:
    
    rows = session.execute(query)
except Exception as e:
    print(e)

### And Finally close the session and cluster connection

In [12]:
session.shutdown()
cluster.shutdown()